# Run the Tracker

This jupyter notebook bundles all necessary high-level steps to train the tracker. The first step is to include all necessary modules.

In [ ]:
from tqdm import tqdm
import pickle
import sys
import numpy as np
from datetime import datetime
import os
sys.path.append('build/')
import simulation
from time import time, sleep
from python.plotter import Plotter
from python.GNNtracker import GNNTracker
from python.constants import *
from python.metrics import BCE

A simple 2D simulation simulates is used to generate the data of the robot perceptions. The next cells runs a new simulation to generate fresh data. The fresh data is saved by a simulation log pickle file. This cell is optional, we can also load the data from an old simulation log.

In [ ]:
T_simulation = 60 * 60 * 2 # 1 hour

sim = simulation.Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
)
sim_states = []
N_minutes = int(T_simulation / 60)
N_hours = int(T_simulation / 3600)
pbar = tqdm(range(0, N_minutes), desc='Simulation')
for i in pbar:
    sim_states += sim.step(int(60 / sim.T_step))
    pbar.set_postfix(
    {'Simulated time': '{:d}:{:02d} of {:d}:{:02d} hours'.format(
        int(i / 60), 
        i % 60, 
        int(T_simulation / 3600), 
        int(T_simulation / 60) % 60
    )})

sim_log = {
        'T_step': sim.T_step,
        'T_simulation': T_simulation,
        'N_humans': sim.N_humans,
        'N_robots': sim.N_robots,
        'sim_states': sim_states,
    }
filename = os.path.join(LOG_FOLDER, 'log_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.pkl')
with open(filename, 'wb') as outp:
    pickle.dump(sim_log, outp, pickle.HIGHEST_PROTOCOL)
    

This cell loads the simulation data from a simulation log. If no fresh data is available, an old simulation log is loaded.

In [ ]:
if "simulation_log" not in locals():
    filename = os.path.join(LOG_FOLDER, 'log_2024-08-16_20-34-51.pkl')
    with open(filename, 'rb') as f:
        sim_log = pickle.load(f)
sim_states = sim_log['sim_states']
T_simulation = sim_log['T_simulation']
T_step = sim_log['T_step']
N_humans = sim_log['N_humans']
N_robots = sim_log['N_robots']
# load simulation to have access to member utility functions
sim = simulation.Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
)

With the simulation data loaded, the actual tracker can be loaded.

In [ ]:
plot = True
record_video = False
realtime = True

tracker = GNNTracker(
    T_step=T_step,
    train=False,
    # use extend_perception implemented in simulation class with C++ for speedup
    extend_perception=sim.extend_perception 
)

if plot: plotter = Plotter(record_frames=record_video)

pbar = tqdm(range(0, len(sim_states)), desc='Simulation')

simulation_time = 0
node_probabilities = []
for i in pbar:
    start = time()

    sim_state = sim_states[i]

    # outer list: robots, inner list: perceived humans for every robot
    robot_perceptions = [{ 
        "ego_position": agent['ego_position'],
        "perceived_humans": agent['perceived_humans'] 
    } for agent in sim_state if agent['type'] == 'robot']

    tracker.add_observation(robot_perceptions)
    node_probabilities.append(tracker.predict())

    if plot: plotter.update(sim_state, node_probabilities[-1])    
    
    if realtime: sleep(max(0, T_step - (time() - start)))

    pbar.set_postfix(
        {'Simulated time': '{:d}:{:02d} of {:d}:{:02d} hours'.format(
            int(simulation_time / 3600), 
            int(simulation_time / 60) % 60, 
            int(T_simulation / 3600), 
            int(T_simulation / 60) % 60
        )})
    
    simulation_time += T_step

if record_video: plotter.create_video(T_step)

The predicted node probabilites for every node (the output of the tracker) is added to the simulation log and the updated simulation log is saved again.

In [ ]:
sim_log["node_probabilities"] = node_probabilities
with open(os.path.join(LOG_FOLDER, filename), 'wb') as outp:
    pickle.dump(sim_log, outp, pickle.HIGHEST_PROTOCOL)

Now we can benchmark the performance of our tracker.

In [ ]:
benchmarker = Benchmarker(sim_log)